In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd

from optses.analysis import analyze_results_lp, analyze_results_nl
from optses.plots import plot_benchmark, plot_power_ecdf, plot_sensitivity

In [ ]:
def load_price_timeseries(file: str) -> pd.Series:
    df = pd.read_csv(file)
    df.index = pd.to_datetime(df["Date"], format="%d/%m/%Y %H:%M")
    return df["Intraday Continuous 15 minutes ID1-Price"]  # €/MWh

In [ ]:
def load_results(dir: str) -> dict[str, pd.DataFrame]:
    res = {}
    files = [f for f in os.listdir(dir) if f.endswith(".parquet")]
    for file in files:
        name, _ = os.path.splitext(file)
        res[name] = pd.read_parquet(dir + file)

    return res

In [ ]:
price = load_price_timeseries("data/intraday_prices/electricity_prices_germany_2021.csv")

In [ ]:
dir = "results/"
res = load_results(dir)

df_lp = analyze_results_lp(res, price)
df_nl = analyze_results_nl(res, price)

In [ ]:
# filter base scenarios
lp_params = [(1.0, 0.959), (2.0, 0.946), (3.0, 0.933)]
df_lp0 = pd.concat([df_lp[(df_lp.r == r) & (df_lp.eff == eff)] for (r, eff) in lp_params])
df_nl0 = df_nl[df_nl.r_opt == 1.0]

In [ ]:
df_lp0.style.format({
    "r": "{:.1f}", # SOH_R scenario
    "eff": "{:.1%}", # efficiency model param (sensitivity scenario)
    "rev": "{:,.0f}", # revenue in €
    "fec": "{:,.0f}", # number of full equivalent cycles
    "rte": "{:.1%}", # roundtrip efficiency
    "loss_batt": "{:.1%}", # relative battery losses
    "loss_conv": "{:.1%}", # relative converter losses
    "imb_ch": "{:,.0f}", # energy imbalance in Wh
    "imb_dch": "{:,.0f}", # energy imbalance in Wh
})

In [ ]:
df_nl0.style.format({
    "r": "{:.1f}", # SOH_R scenario
    "r_opt": "{:.1f}", # SOH_R model, deviation factor (sensitivity scenario)
    "rev": "{:,.0f}", # revenue in €
    "fec": "{:.0f}", # number of full equivalent cycles
    "rte": "{:.1%}", # roundtrip efficiency
    "loss_batt": "{:.1%}", # relative battery losses
    "loss_conv": "{:.1%}", # relative converter losses
    "imb_ch": "{:,.0f}", # energy imbalance in Wh
    "imb_dch": "{:,.0f}", # energy imbalance in Wh
})

In [ ]:
fig1 = plot_benchmark(df_lp0, df_nl0)

In [ ]:
fig2 = plot_power_ecdf(
    res_lp = {r: res[f"2021 LP {r=} {eff=}"] for r, eff in lp_params},
    res_nl = {r: res[f"2021 NL {r=} r_opt=1.0"] for r in (1.0, 2.0, 3.0)}
)

In [ ]:
fig3 = plot_sensitivity(df_lp, df_nl, df_lp0, df_nl0)

In [ ]:
fig1.savefig("figs/benchmark.pdf", bbox_inches="tight")
fig2.savefig("figs/power_dist.pdf", bbox_inches="tight")
fig3.savefig("figs/sensitivity.pdf", bbox_inches="tight")